# Data Requests 

## Setup

In [1]:
# Import modules
import time

import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys


In [2]:
# API connexion
client_id = 'cec8979c027344f98b471a991aa415ad'
client_secret = 'ad69bcba55b349f98c1344b006c708bd'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


## Get playlists infos 

In [ ]:
# API scrapping
# Set category IDs
category_ids = [
    '0JQ5DAqbMKFNQ0fGp4byGU', '0JQ5DAqbMKFx0uLQR2okcc', '0JQ5DAqbMKFFtlLYUHv8bT', '0JQ5DAqbMKFzHmL4tf05da', '0JQ5DAqbMKFLjmiZRss79w',
    '0JQ5DAqbMKFAUsdyVjCQuL', '0JQ5DAqbMKFziKOShCi009', '0JQ5DAqbMKFQ1UFISXj59F', '0JQ5DAqbMKFSCjnQr8QZ3O', '0JQ5DAqbMKFLb2EqgLtpjC', 
    '0JQ5DAqbMKFQiK2EHwyjcU', '0JQ5DAqbMKFObNLOHydSW8', '0JQ5DAqbMKFPrEiAOxgac3', '0JQ5DAqbMKFKLfwjuJMoNC', '0JQ5DAqbMKFRY5ok2pxXJ0', 
    '0JQ5DAqbMKFHOzuVTgTizF', '0JQ5DAqbMKFP9UNkavIGrM', '0JQ5DAqbMKFIVNxQgRNSg0', '0JQ5DAqbMKFz6FAsUtgAab', '0JQ5DAqbMKFFzDl7qN9Apr', 
    '0JQ5DAqbMKFCuoRTxhYWow', '0JQ5DAqbMKFFoimhOqWzLB', '0JQ5DAqbMKFIRybaNTYXXy', '0JQ5DAqbMKFPw634sFwguI', '0JQ5DAqbMKFJw7QLnM27p6', 
    '0JQ5DAqbMKFCbimwdOYlsl', '0JQ5DAqbMKFy78wprEpAjl', '0JQ5DAqbMKFF9bY76LXmfI', '0JQ5DAqbMKFImHYGo3eTSg', '0JQ5DAqbMKFFsW9N8maB6z', 
    '0JQ5DAqbMKFCfObibaOZbv', '0JQ5DAqbMKFGnsSfvg90Wo', '0JQ5DAqbMKFQ00XGBls6ym', '0JQ5DAqbMKFCWjUTdzaG0e', '0JQ5DAqbMKFRKBHIxJ5hMm', 
    '0JQ5DAqbMKFAJ5xb0fwo9m', '0JQ5DAqbMKFGvOw3O4nLAf', '0JQ5DAqbMKFxXaXKP7zcDp', '0JQ5DAqbMKFDkd668ypn6O', '0JQ5DAqbMKFRieVZLLoo9m', 
    '0JQ5DAqbMKFI3pNLtYMD9S', '0JQ5DAqbMKFEOEBCABAxo9', '0JQ5DAqbMKFEC4WFtoNRpw', '0JQ5DAqbMKFAjfauKLOZiv', '0JQ5DAqbMKFOOxftoKZxod', 
    '0JQ5DAqbMKFJKoGyUMo2hE', '0JQ5DAqbMKFEZPnFQSFB1T', '0JQ5DAqbMKFDXXwE9BDJAr', '0JQ5DAqbMKFA6SOHvT3gck', '0JQ5DAqbMKFIpEuaCnimBj', 
    '0JQ5DAqbMKFAXlCG6QvYQ4', '0JQ5DAqbMKFDBgllo2cUIN', '0JQ5DAqbMKFQIL0AXnG5AK', '0JQ5DAqbMKFOzQeOmemkuw', '0JQ5DAqbMKFBweu2euf3Sj', 
    '0JQ5DAqbMKFAQy4HL4XU2D'
]

# Store missing categories
missing_categories = []

# Loop over all categories and get playlists
playlists = []
for category_id in category_ids:
    try:
        # Get first 50 playlists in category
        response = sp.category_playlists(category_id, limit=50)
        category_playlists = response["playlists"]["items"]
        
        # Check if there are more playlists in category
        while response["playlists"]["next"]:
            response = sp.next(response["playlists"])
            category_playlists += response["playlists"]["items"]
        
        # Append category playlists to list
        for playlist in category_playlists:
            playlist_id = playlist["id"]
            playlist_name = playlist["name"]
            playlist_owner = playlist["owner"]["display_name"]
            playlists.append({
                "category_id": category_id, 
                "playlist_id": playlist_id, 
                "playlist_name": playlist_name,
                "playlist_owner": playlist_owner
            })
    except: 
        missing_categories.append(category_id)
        print(f"Category with ID {category_id} doesn't exist.")

# Create dataframe from playlists
df_playlists = pd.DataFrame(playlists, columns=["category_id", "playlist_id", "playlist_name", "playlist_owner"])

print(df_playlists)


In [ ]:
# HTML scraping for Charts playlists
# URLs to scrape
urls = [
    'https://open.spotify.com/genre/section0JQ5DAzQHECxDlYNI6xD1h',
    'https://open.spotify.com/genre/section0JQ5DAzQHECxDlYNI6xD1i',
    'https://open.spotify.com/genre/section0JQ5DAzQHECxDlYNI6xD1j'
]

# Set up the web driver
driver = webdriver.Safari(executable_path='/usr/bin/safaridriver')

top_playlists = []

for url in urls:
    driver.get(url)
    time.sleep(5)  # Wait for playlists to load

    # Find playlist elements by class name
    playlists = driver.find_elements(By.CLASS_NAME, 'Box__BoxComponent-sc-y4nds-0')
    last_playlist = playlists[-1]

    # Scroll to load all playlists
    while True:
        driver.execute_script("arguments[0].scrollIntoView();", last_playlist)
        time.sleep(2)
        new_playlists = driver.find_elements(By.CLASS_NAME, 'Box__BoxComponent-sc-y4nds-0')
        if new_playlists[-1] != last_playlist:
            last_playlist = new_playlists[-1]
            playlists = new_playlists
        else:
            break

    # Extract playlist information
    for playlist in playlists:
        # Get aria-labelledby attribute
        aria_label = playlist.get_attribute('aria-labelledby')
        
        # Extract playlist ID from aria-labelledby
        if aria_label and "spotify:playlist:" in aria_label:
            playlist_id = aria_label.split("spotify:playlist:")[1].split()[0]
            top_playlists.append({
                "category_id": '0JQ5DAudkNjCgYMM0TZXDw', 
                "playlist_id": playlist_id, 
                "playlist_owner": 'Spotify'
            })

# Quit the driver
driver.quit()

# Convert data to DataFrame and print
df_top_playlists = pd.DataFrame(top_playlists, columns=["category_id", "playlist_id", "playlist_owner"])
print(df_top_playlists)


In [11]:
# Parse playlists info into excel
# Combine the manual and API playlists into one DataFrame
df_combined = pd.concat([df_playlists, df_top_playlists], ignore_index=True)
df_combined.to_excel("playlists_info.xlsx", index=False)

# Drop duplicates based on the "playlist_id" column, keeping the first occurrence
df_cleaned = df_combined.drop_duplicates(subset='playlist_id', keep='first')
df_cleaned.to_excel("playlists_info_clean.xlsx", index=False)


## Get tracks info

### Setup

In [3]:
# Load data
df = pd.read_excel('playlists_info_clean.xlsx')

# Remove playlists with "This is" in the name
df = df[~df['playlist_name'].str.contains('This Is', na=False)]


In [4]:
# Define scraping functions
# Define a function to get all the tracks in a playlist, handling the API limit
def get_all_playlist_tracks(playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

# Define a function to get all the audio features for a set of tracks, handling the API limit
def get_all_audio_features(track_ids):
    audio_features = []
    for i in range(0, len(track_ids), 100):
        batch = track_ids[i:i+100]
        try:
            features = sp.audio_features(batch)
        except:
            continue
        audio_features.extend(features)
    return audio_features

def get_playlist_track_info(playlist_id):
    # Get the playlist info
    playlist_info = sp.playlist(playlist_id)
    if not playlist_info:
        return None
    playlist_name = playlist_info['name']
    playlist_followers = playlist_info['followers']['total']
    playlist_owner = playlist_info['owner']['display_name']
    
    # Get all the tracks in the playlist
    playlist_tracks = get_all_playlist_tracks(playlist_id)
    track_ids = [track['track']['id'] for track in playlist_tracks if track['track'] is not None]
    
    if not track_ids:
        return None
    
    # Get the track info in batches of 50
    track_info = []
    for i in range(0, len(track_ids), 50):
        batch_tracks = track_ids[i:i+50]
        tracks = sp.tracks(batch_tracks)['tracks']
        track_info.extend(tracks)
    
    track_names = [track['name'] for track in track_info]
    track_popularities = [track['popularity'] for track in track_info]
    track_release_dates = [track['album']['release_date'] for track in track_info]
    artist_ids = [track['artists'][0]['id'] for track in track_info]
    
    artist_info = []
    for i in range(0, len(artist_ids), 50):
        batch_artists = artist_ids[i:i+50]
        artists = sp.artists(batch_artists)['artists']
        artist_info.extend(artists)
    
    artist_names = [artist['name'] for artist in artist_info]
    artist_popularities = [artist['popularity'] for artist in artist_info]

    # Combine everything into a DataFrame
    playlist_df = pd.DataFrame({
        'playlist_name': playlist_name,
        'playlist_followers': playlist_followers,
        'playlist_owner': playlist_owner,
        'track_name': track_names,
        'track__popularity': track_popularities,
        'track_release_date': track_release_dates,
        'artist_name': artist_names,
        'artist_popularity': artist_popularities
    })

    # Get the audio features
    audio_features = get_all_audio_features(track_ids)
    if audio_features is None:
        return None
    
    # Add the audio features to the DataFrame
    try:
        audio_features_df = pd.DataFrame(audio_features)
        playlist_df = pd.concat([playlist_df, audio_features_df], axis=1)
    except:
        return None
    
    return playlist_df

# Define a function to loop over all the playlists and combine the results
def get_all_playlist_track_info(playlist_ids):
    playlist_dfs = []
    for playlist_id in playlist_ids:
        print(f"Processing playlist: {playlist_id}")
        playlist_df = get_playlist_track_info(playlist_id)
        if playlist_df is not None:
            playlist_dfs.append(playlist_df)
    all_playlist_df = pd.concat(playlist_dfs, ignore_index=True)
    return all_playlist_df


### Scrape the data for each category

In [5]:
# Tops
category_id = "0JQ5DAudkNjCgYMM0TZXDw"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_tops = get_all_playlist_track_info(playlist_ids)
df_tops.to_excel("df_tops.xlsx", index = False)


Processing playlist: 37i9dQZEVXbNG2KDcFcKOF
Processing playlist: 37i9dQZEVXbJV3H3OfCN1z
Processing playlist: 37i9dQZEVXbK8BKKMArIyl
Processing playlist: 37i9dQZEVXbO839WGRmpu1
Processing playlist: 37i9dQZEVXbKPTKrnFPD0G
Processing playlist: 37i9dQZEVXbK4fwx2r07XW
Processing playlist: 37i9dQZEVXbM1EaZ0igDlz
Processing playlist: 37i9dQZEVXbND4ZYa46PaA
Processing playlist: 37i9dQZEVXbLRLeF2cVSaP
Processing playlist: 37i9dQZEVXbJgjCMawPquO
Processing playlist: 37i9dQZEVXbKzoK95AbRy9
Processing playlist: 37i9dQZEVXbJQbYjwxrhPm
Processing playlist: 37i9dQZEVXbMda2apknTqH
Processing playlist: 37i9dQZEVXbLJ0paT1JkgZ
Processing playlist: 37i9dQZEVXbKVvIaSFCnNP
Processing playlist: 37i9dQZEVXbL1Fl8vdBUba
Processing playlist: 37i9dQZEVXbJZGli0rRP3r
Processing playlist: 37i9dQZEVXbNDIQm4XMct1
Processing playlist: 37i9dQZEVXbMw2iUtFR5Eq
Processing playlist: 37i9dQZEVXbJwoKy8qKpHG
Processing playlist: 37i9dQZEVXbLnGyU4lLfIR
Processing playlist: 37i9dQZEVXbJQ9kF73GOT2
Processing playlist: 37i9dQZEVXb

In [6]:
# Afro
category_id = "0JQ5DAqbMKFNQ0fGp4byGU"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_pop = get_all_playlist_track_info(playlist_ids)
df_pop.to_excel("df_afro.xlsx", index = False)


Processing playlist: 37i9dQZF1DWX0o6sD1a6P5
Processing playlist: 37i9dQZF1DX6036iaZ2MYP
Processing playlist: 37i9dQZF1DX3oRZ26ZzwxF
Processing playlist: 37i9dQZF1DWYkaDif7Ztbp
Processing playlist: 37i9dQZF1DX41PoyDLE3r1
Processing playlist: 37i9dQZF1DX0z1epx5KTtS
Processing playlist: 37i9dQZF1DWT0W2NVqAFjU
Processing playlist: 37i9dQZF1DWYs2pvwxWA7l
Processing playlist: 37i9dQZF1DWSjibQnF0bUk
Processing playlist: 37i9dQZF1DX8d57cbyhqPM
Processing playlist: 37i9dQZF1DX117wqItTnqW
Processing playlist: 37i9dQZF1DWUS3jbm4YExP
Processing playlist: 37i9dQZF1DWT6SJaitNDax
Processing playlist: 37i9dQZF1DX1l6qs3gcM4U
Processing playlist: 37i9dQZF1DX6eyIKoZCdug
Processing playlist: 37i9dQZF1DWXRzAsbxGrBZ
Processing playlist: 37i9dQZF1DWX4IFcj5utrY
Processing playlist: 37i9dQZF1DWY7SO3HC2gxu
Processing playlist: 37i9dQZF1DX5mILnBJLA26
Processing playlist: 37i9dQZF1DXd9mvqWzJEWg
Processing playlist: 37i9dQZF1DXb6pBzjNjXR1
Processing playlist: 37i9dQZF1DWTmJfgGWCsDY
Processing playlist: 37i9dQZF1DX

In [7]:
# Maison
category_id = "0JQ5DAqbMKFx0uLQR2okcc"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_maison = get_all_playlist_track_info(playlist_ids)
df_maison.to_excel("df_maison.xlsx", index = False)


Processing playlist: 37i9dQZF1DWSM0Vq39VM1S
Processing playlist: 37i9dQZF1DWUMDyKd0qQIL
Processing playlist: 37i9dQZF1DX0X277P8nU0l
Processing playlist: 37i9dQZF1DX42c5TkSUfEb
Processing playlist: 37i9dQZF1DX2jdUWwSgTho
Processing playlist: 37i9dQZF1DX5wSQbrOxKS2
Processing playlist: 37i9dQZF1DWUF3bcja4I8P
Processing playlist: 37i9dQZF1DWXGPruVsfHKD
Processing playlist: 37i9dQZF1DWY77tub05nHz
Processing playlist: 37i9dQZF1DWZk2SPZ3bugX
Processing playlist: 37i9dQZF1DXbsWQD03AVcA
Processing playlist: 37i9dQZF1DWZa8CSUr0hCY
Processing playlist: 37i9dQZF1DWT5lkChsPmpy
Processing playlist: 37i9dQZF1DWZeKCadgRdKQ
Processing playlist: 37i9dQZF1DX1uaml3UXYLk
Processing playlist: 37i9dQZF1DWUZ5bk6qqDSy
Processing playlist: 37i9dQZF1DWZIOAPKUdaKS
Processing playlist: 37i9dQZF1DX1S1NduGwpsa
Processing playlist: 37i9dQZF1DWXLeA8Omikj7
Processing playlist: 37i9dQZF1DWWQRwui0ExPn
Processing playlist: 37i9dQZF1DX742okrrpwah
Processing playlist: 37i9dQZF1DWUSKuTscFFGY
Processing playlist: 37i9dQZF1DW

In [8]:
# Alternative
category_id = "0JQ5DAqbMKFFtlLYUHv8bT"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_alternative = get_all_playlist_track_info(playlist_ids)
df_alternative.to_excel("df_alternative.xlsx", index = False)


Processing playlist: 37i9dQZF1DX4TqmALHVh5A
Processing playlist: 37i9dQZF1DWSluyUi4YGjq
Processing playlist: 37i9dQZF1DXa71eg5j9dKZ
Processing playlist: 37i9dQZF1DX98f0uoU1Pcs
Processing playlist: 37i9dQZF1DXciq74vrQmM8
Processing playlist: 37i9dQZF1DWXMg4uP5o3dm
Processing playlist: 37i9dQZF1DXd9H435blNqg
Processing playlist: 37i9dQZF1DWTMR78LDoAZC
Processing playlist: 37i9dQZF1DWY5ai7gxfuaS
Processing playlist: 37i9dQZF1DX9GRpeH4CL0S
Processing playlist: 37i9dQZF1DX873GaRGUmPl
Processing playlist: 37i9dQZF1DX0YKekzl0blG
Processing playlist: 37i9dQZF1DXaLI8a395lse
Processing playlist: 37i9dQZF1DWTSKFpOdYF1r
Processing playlist: 37i9dQZF1DXb3ZjVksUlfu
Processing playlist: 37i9dQZF1DX5qNE4zrflL7
Processing playlist: 37i9dQZF1DXdTCdwCKzXwo
Processing playlist: 37i9dQZF1DX0Nb1cyWS6Tq
Processing playlist: 37i9dQZF1DWSAm0NxvFu7q
Processing playlist: 37i9dQZF1DXcF6B6QPhFDv
Processing playlist: 37i9dQZF1DX18m1EYRUeh9
Processing playlist: 37i9dQZF1DX82GYcclJ3Ug
Processing playlist: 37i9dQZF1DW

In [ ]:
category_id = "0JQ5DAqbMKFxXaXKP7zcDp" # Latino

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_latino = get_all_playlist_track_info(playlist_ids)

df_latino.to_excel("df_latino.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFzHmL4tf05da" # Ambiance

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_ambiance = get_all_playlist_track_info(playlist_ids)

df_ambiance.to_excel("df_ambiance.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFHOzuVTgTizF" # Electro

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_electro = get_all_playlist_track_info(playlist_ids)

df_electro.to_excel("df_electro.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFDXXwE9BDJAr" # Rock

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_rock = get_all_playlist_track_info(playlist_ids)

df_rock.to_excel("df_rock.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFCWjUTdzaG0e" # Indie

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_indie = get_all_playlist_track_info(playlist_ids)

df_indie.to_excel("df_indie.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFAXlCG6QvYQ4" # Sport

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_sport = get_all_playlist_track_info(playlist_ids)

df_sport.to_excel("df_sport.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFEZPnFQSFB1T" # RnB

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_rnb = get_all_playlist_track_info(playlist_ids)

df_rnb.to_excel("df_rnb.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFNQ0fGp4byGU" # Afro

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_afro = get_all_playlist_track_info(playlist_ids)

df_afro.to_excel("df_afro.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFGvOw3O4nLAf" # K-pop

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_kpop = get_all_playlist_track_info(playlist_ids)

df_kpop.to_excel("df_kpop.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFFzDl7qN9Apr" # Détente

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_detente = get_all_playlist_track_info(playlist_ids)

df_detente.to_excel("df_detente.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFCuoRTxhYWow" # Dormir

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_dormir = get_all_playlist_track_info(playlist_ids)

df_dormir.to_excel("df_dormir.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFA6SOHvT3gck" # Soirée

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_soiree = get_all_playlist_track_info(playlist_ids)

df_soiree.to_excel("df_soiree.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFP9UNkavIGrM" # Zouk

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_zouk = get_all_playlist_track_info(playlist_ids)

df_zouk.to_excel("df_zouk.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFJKoGyUMo2hE" # Reggae

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_reggae = get_all_playlist_track_info(playlist_ids)

df_reggae.to_excel("df_reggae.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFx0uLQR2okcc" # A la maison

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_maison = get_all_playlist_track_info(playlist_ids)

df_maison.to_excel("df_maison.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFIVNxQgRNSg0" # Decennies

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_decennies = get_all_playlist_track_info(playlist_ids)

df_decennies.to_excel("df_decennies.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFFtlLYUHv8bT" # Alternatif

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_alternatif = get_all_playlist_track_info(playlist_ids)

df_alternatif.to_excel("df_alternatif.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFAUsdyVjCQuL" # Amour

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_amour = get_all_playlist_track_info(playlist_ids)

df_amour.to_excel("df_amour.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFDkd668ypn6O" # Metal

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_metal = get_all_playlist_track_info(playlist_ids)

df_metal.to_excel("df_metal.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFAJ5xb0fwo9m" # Jazz

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_jazz = get_all_playlist_track_info(playlist_ids)

df_jazz.to_excel("df_jazz.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFQIL0AXnG5AK" # Tendances

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_tendances = get_all_playlist_track_info(playlist_ids)

df_tendances.to_excel("df_tendances.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFPw634sFwguI" # Equal

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_equal = get_all_playlist_track_info(playlist_ids)

df_equal.to_excel("df_equal.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFPrEiAOxgac3" # Classique

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_classique = get_all_playlist_track_info(playlist_ids)

df_classique.to_excel("df_classique.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFy78wprEpAjl" # Folk

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_folk = get_all_playlist_track_info(playlist_ids)

df_folk.to_excel("df_folk.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFCbimwdOYlsl" # Focus

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_focus = get_all_playlist_track_info(playlist_ids)

df_focus.to_excel("df_focus.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFIpEuaCnimBj" # Soul

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_soul = get_all_playlist_track_info(playlist_ids)

df_soul.to_excel("df_soul.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFEOEBCABAxo9" # Netflix

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_netflix = get_all_playlist_track_info(playlist_ids)

df_netflix.to_excel("df_netflix.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFFoimhOqWzLB" # Enfants

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_enfants = get_all_playlist_track_info(playlist_ids)

df_enfants.to_excel("df_enfants.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFJw7QLnM27p6" # Etudiants

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_etudiants = get_all_playlist_track_info(playlist_ids)

df_etudiants.to_excel("df_etudiants.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFLb2EqgLtpjC" # Bien-être

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_bienetre = get_all_playlist_track_info(playlist_ids)

df_bienetre.to_excel("df_bienetre.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFAjfauKLOZiv" # Punk

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_punk = get_all_playlist_track_info(playlist_ids)

df_punk.to_excel("df_punk.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFLjmiZRss79w" # Ambient

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_ambient = get_all_playlist_track_info(playlist_ids)

df_ambient.to_excel("df_ambient.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFKLfwjuJMoNC" # Country

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_country = get_all_playlist_track_info(playlist_ids)

df_country.to_excel("df_country.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFQiK2EHwyjcU" # Blues

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_blues = get_all_playlist_track_info(playlist_ids)

df_blues.to_excel("df_blues.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFQ1UFISXj59F" # Arabe

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_arabe = get_all_playlist_track_info(playlist_ids)

df_arabe.to_excel("df_arabe.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFAQy4HL4XU2D" # Voyage

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_voyage = get_all_playlist_track_info(playlist_ids)

df_voyage.to_excel("df_voyage.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFF9bY76LXmfI" # Frequency

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_frequency = get_all_playlist_track_info(playlist_ids)

df_frequency.to_excel("df_frequency.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAudkNjCgYMM0TZXDr" # Tops

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_tops = get_all_playlist_track_info(playlist_ids)

df_tops.to_excel("df_tops.xlsx", index = False)
